Ejemplo de:https://www.youtube.com/watch?v=p2sTJYoIwj0&t=53s&ab_channel=CodificandoBits


In [1]:
pip install keras-transformer

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12286 sha256=8eee28ec93a46f3bd60c693ad18dd2a72061ca7f4bdcd8c0daee11b1c108a7d2
  Stored in directory: /root/.cache/pip/wheels/f2/cb/22/75a0ad376129177f7c95c0d91331a18f5368fd657f4035ba7c
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3944 sha256=1a08737118b30133b1313a27d38056fce82fa1a43bbe5f6edc8b48460bbcb4f9
  Stored in directory: /root/.cache/pip/wheels/82/32/c7/fd35d0d1b840a6c7cbd4343f808d10d0f7b87d271a4dbe796f
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4654 sha256=fbe9a1ecacfde48241c24e5f0b2e6077617

In [2]:
# Import libraries
import numpy as np
from keras_transformer import get_model, decode
from pickle import load
from google.colab import drive
np.random.seed(0)

In [3]:
# Load data and print examples
drive.mount('/content/drive')
filename = '/content/drive/MyDrive/english-spanish.pkl'

dataset = load(open(filename, 'rb'))
print(dataset[120000,0])
print(dataset[120000,1])


Mounted at /content/drive
tom is a new yorker but he doesnt have a new york accent
tom es neoyorquino pero no tiene acento de nueva york


In [4]:
# Make tokens, this is divide string in words
source_tokens = []
for sentence in dataset[:,0]:
  # Column 0, english phrases
  source_tokens.append(sentence.split(' '))
print(source_tokens[120000])

target_tokens = []
for sentence in dataset[:,1]:
   # Column 1, spanish phrases
  target_tokens.append(sentence.split(' '))
print(target_tokens[120000])

['tom', 'is', 'a', 'new', 'yorker', 'but', 'he', 'doesnt', 'have', 'a', 'new', 'york', 'accent']
['tom', 'es', 'neoyorquino', 'pero', 'no', 'tiene', 'acento', 'de', 'nueva', 'york']


In [5]:
# Create dictionaries, this is emmbeding, transform tokens in numbers

def build_token_dict(token_list):
  token_dict = {
      '': 0,
      '': 1,
      '': 2
  }
  for tokens in token_list:
    for token in tokens:
      if token not in token_dict:
        token_dict[token] = len(token_dict)
  return token_dict


source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
target_token_dict_inv = {v:k for k,v in target_token_dict.items()}
# Dict_inv is to use in decoder.

print(source_token_dict)
print(target_token_dict)
print(target_token_dict_inv)



{'': 2, 'go': 1, 'hi': 2, 'run': 3, 'who': 4, 'wow': 5, 'fire': 6, 'help': 7, 'jump': 8, 'stop': 9, 'wait': 10, 'on': 11, 'hello': 12, 'hurry': 13, 'i': 14, 'hid': 15, 'ran': 16, 'try': 17, 'won': 18, 'oh': 19, 'no': 20, 'relax': 21, 'shoot': 22, 'smile': 23, 'attack': 24, 'get': 25, 'up': 26, 'now': 27, 'got': 28, 'it': 29, 'he': 30, 'hop': 31, 'in': 32, 'hug': 33, 'me': 34, 'care': 35, 'fell': 36, 'fled': 37, 'know': 38, 'left': 39, 'lied': 40, 'lost': 41, 'quit': 42, 'sang': 43, 'wept': 44, 'work': 45, 'im': 46, 'listen': 47, 'way': 48, 'really': 49, 'thanks': 50, 'we': 51, 'why': 52, 'ask': 53, 'tom': 54, 'awesome': 55, 'be': 56, 'calm': 57, 'cool': 58, 'fair': 59, 'good': 60, 'kind': 61, 'nice': 62, 'beat': 63, 'call': 64, 'us': 65, 'come': 66, 'drop': 67, 'out': 68, 'away': 69, 'home': 70, 'slow': 71, 'goodbye': 72, 'hang': 73, 'came': 74, 'hit': 75, 'hold': 76, 'agree': 77, 'bowed': 78, 'fired': 79, 'froze': 80, 'moved': 81, 'slept': 82, 'tried': 83, 'ill': 84, 'fat': 85, 'fit':

In [6]:
# Add start, end and pad to training set.
encoder_tokens = [[''] + tokens + [''] for tokens in source_tokens]
decoder_tokens = [[''] + tokens + [''] for tokens in target_tokens]
output_tokens = [tokens + [''] for tokens in target_tokens]

# Pad uses the max of the phrases english and spanish
source_max_len = max(map(len, encoder_tokens))
target_max_len = max(map(len, decoder_tokens))

# Add pading
encoder_tokens = [tokens + ['']*(source_max_len-len(tokens)) for tokens in encoder_tokens]
decoder_tokens = [tokens + ['']*(target_max_len-len(tokens)) for tokens in decoder_tokens]
output_tokens = [tokens + ['']*(target_max_len-len(tokens)) for tokens in output_tokens ]


print(encoder_tokens[120000])



['', 'tom', 'is', 'a', 'new', 'yorker', 'but', 'he', 'doesnt', 'have', 'a', 'new', 'york', 'accent', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [7]:
# Using dictionaries convert character to number
encoder_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encoder_tokens]
decoder_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decoder_tokens]
output_decoded = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

# Show character in number, this is an example
print(encoder_input[120000])


[2, 54, 256, 118, 195, 12664, 2912, 30, 1575, 138, 118, 195, 5383, 4285, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [8]:
# Build a transfomer model

model = get_model(
    # Using keras
    token_num = max(len(source_token_dict),len(target_token_dict)),
    # Max of tokens
    embed_dim = 32,
    # Elements in embeding ¿why 32?
    encoder_num = 2,
    # Numbers of encoders, in original model uses 6, this is more complex and so uses 2
    decoder_num = 2,
    # Numbers of decodes, ibid
    head_num = 4,
    # Attention blocks. In paper uses 8. People watching the prhases
    hidden_dim = 128,
    # neurons y MLP that is a hidden layer.
    dropout_rate = 0.05,
    # % of neurons that desactivate in each epoc of training. Evitate overfiting.
    use_same_embed = False,
    # The embeding is diferent in coder and decoder block.
)
model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()
token_num = max(len(source_token_dict),len(target_token_dict))
print(token_num)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Encoder-Input (InputLayer)  [(None, None)]               0         []                            
                                                                                                  
 Encoder-Token-Embedding (E  [(None, None, 32),           808544    ['Encoder-Input[0][0]']       
 mbeddingRet)                 (25267, 32)]                                                        
                                                                                                  
 Encoder-Embedding (TrigPos  (None, None, 32)             0         ['Encoder-Token-Embedding[0][0
 Embedding)                                                         ]']                           
                                                                                              

In [9]:
# Training

x = [np.array(encoder_input), np.array(decoder_input)]
# English phrases, spanish phrases. This are inputs
y = np.array(output_decoded)
# This is output. The traduction

In [10]:
# Fit model

model.fit(x,y, epochs=1, batch_size=32)
# Batch represents blocks in each epoch



3886/3886 [==============================] - 190s 42ms/step - loss: 0.8260


In [15]:
# This is the function to translate

def translate(sentence):
  sentence_tokens = [tokens + ['', ''] for tokens in [sentence.split(' ')]]
  tr_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in sentence_tokens][0]
  decoded = decode(
      model,
      tr_input,
      start_token = target_token_dict[''],
      end_token = target_token_dict[''],
      pad_token = target_token_dict['']
  )

  print('Frase original: {}'.format(sentence))
  print('Traducción: {}'.format(' '.join(map(lambda x: target_token_dict_inv[x], decoded[1:-1]))))


In [17]:
# an example
translate('this is the house')


1/1 [==============================] - 0s 25ms/step
Frase original: this is the house
Traducción: el es que su su su coche de su su su un coche un coche de mi coche un un un un un un un un un
